<a href="https://colab.research.google.com/github/hailusong/nlp-qa/blob/master/nlp_textsum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### **Text Summarization**
Hyper-params:
TBA

Issues:
1. Max sentence length: 64 (truncated)

## Environment Setup
To get the source code of Huggingface transformer release, do this (assuming the release version is 2.11.0)
```
git fetch -a --tags
git checkout tags/v2.11.0
```

Make sure we have access to file command in Linux

In [2]:
!uname -a
!pip install wget
!apt-get install file

Linux df99a98cd6bd 4.19.104+ #1 SMP Wed Feb 19 05:26:34 PST 2020 x86_64 x86_64 x86_64 GNU/Linux
  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=f020b90206b1d2601b10ff5092f2d0609fd14bf6aa4150962fff0925faccf16f
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libmagic-mgc libmagic1
The following NEW packages will be installed:
  file libmagic-mgc libmagic1
0 upgraded, 3 newly installed, 0 to remove and 32 not upgraded.
Need to get 275 kB of archives.
After this operation, 5,297 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic-mgc amd64 1:5.32-2ubuntu0.4 [184 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic1 amd64 1:5.32-2ubuntu0.4 [68

In [3]:
try:
    import transformers
except ImportError as e:
    # install huggingface
    !pip install transformers

     |████████████████████████████████| 675kB 8.3MB/s 
     |████████████████████████████████| 890kB 24.5MB/s 
     |████████████████████████████████| 1.1MB 50.5MB/s 
     |████████████████████████████████| 3.8MB 52.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=cf6ea8fcf9e654c169f310f380dc45e7442f4395e9db6b467ac573dbd5a1d464
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [4]:
import torch
import transformers

print(torch.__version__)
print(transformers.__version__)

1.5.0+cu101
2.11.0


## Configuratiuon

In [0]:
EXAMPLE_MODE=True
INLINE_MODE=False

In [6]:
if EXAMPLE_MODE:
  # ! [ ! -d "./transformers" ] && git clone --depth 1 https://github.com/huggingface/transformers
  # ! [ -d "./transformers" ] && \
  #   cd transformers && \
  #   git fetch -a --tags && \
  #   git checkout tags/v2.11.0 && \
  #   pip install . && \
  !rm -rf ./nlp-qa/
  ![ ! -d "./nlp-qa" ] && git clone --depth 1 https://github.com/hailusong/nlp-qa.git
  !pip install nltk py-rouge

Cloning into 'nlp-qa'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 36 (delta 2), reused 28 (delta 2), pack-reused 0
Unpacking objects: 100% (36/36), done.
     |████████████████████████████████| 61kB 4.3MB/s 


### GPU Setup

In [7]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


## Download

### Pre-trained BERTABS fine-tuned on CNN/DM corpus 


In [0]:
if INLINE_MODE:
  # from transformers import AutoTokenizer, AutoModelForQuestionAnswering
  # from transformers import AlbertConfig, AlbertForSequenceClassification
  # import torch
  from modeling_bertabs import BertAbs, build_predictor
  from transformers import BertTokenizer

  tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)
  model = BertAbs.from_pretrained("bertabs-finetuned-cnndm")
  model.to(args.device)
  model.eval()

### Text to be summarized

In [17]:
if EXAMPLE_MODE:
  %env DATA_PATH="/content/textdata"
  %env SUMMARIES_PATH="/content/textdata"

  !mkdir -p $DATA_PATH
  !ls -al $DATA_PATH


env: DATA_PATH="/content/textdata"
env: SUMMARIES_PATH="/content/textdata"
total 8
drwxr-xr-x 2 root root 4096 Jun  9 21:53 .
drwxr-xr-x 1 root root 4096 Jun  9 21:53 ..


## Inference
Source: https://pytorch.org/hub/huggingface_pytorch-transformers/


In [18]:
if EXAMPLE_MODE:
  # !pwd
  # !ls -al transformers/examples/summarization/bertabs
  # !cd transformers/examples/summarization/bertabs && python run_summarization.py \
  #   --documents_dir $DATA_PATH \
  #   --summaries_output_dir $SUMMARIES_PATH \
  #   --no_cuda false \
  #   --batch_size 4 \
  #   --min_length 50 \
  #   --max_length 200 \
  #   --beam_size 5 \
  #   --alpha 0.95 \
  #   --block_trigram true
  # !ls -al nlp-qa/textsum/bertabs
  !cd nlp-qa/textsum/bertabs && python run_summarization.py \
    --documents_dir $DATA_PATH \
    --summaries_output_dir $SUMMARIES_PATH \
    --no_cuda false \
    --batch_size 4 \
    --min_length 50 \
    --max_length 200 \
    --beam_size 5 \
    --alpha 0.95 \
    --block_trigram true

"/content/textdata"
2020-06-09 21:55:30.445708: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Traceback (most recent call last):
  File "run_summarization.py", line 324, in <module>
    main()
  File "run_summarization.py", line 305, in main
    f"We could not find the directory {args.documents_dir} you specified for the documents to summarize, or it was empty. Please specify a valid path."
FileNotFoundError: We could not find the directory "/content/textdata" you specified for the documents to summarize, or it was empty. Please specify a valid path.
